In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from static_grader import grader

# NLP Miniproject

## Introduction

The objective of this miniproject is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the result of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [2]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [3]:
import gzip
import ujson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [4]:
import numpy as np

overall_array = np.asarray([dictionary.get("overall") for dictionary in data])

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [1]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

In [8]:
key = 'reviewText'

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        list_of_words = []
        for dictionary in X:
            list_of_words.append(dictionary.get(self.key))
        return list_of_words
            

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import STOP_WORDS
import spacy

nlp = spacy.load('en')

def lemmatizer(text):
    return [word.lemma_ for word in nlp(text)]

# we need to generate the lemmas of the stop words
stop_words_str = " ".join(STOP_WORDS) # nlp function needs a string
stop_words_lemma = set(word.lemma_ for word in nlp(stop_words_str))

countVectorizer = CountVectorizer(stop_words=stop_words_lemma,
                                 tokenizer=lemmatizer)

In [10]:
from sklearn.linear_model import Ridge

bag_of_words_model = Pipeline([
    ('extract', ColumnSelectTransformer(key)),
    ('countVec', countVectorizer),
    ('')
    ('ridge', Ridge())
])

In [11]:
#bag_of_words_model.fit(data, overall_array)

In [28]:
#grader.score.nlp__bag_of_words_model(bigrams_model.predict)

Your score:  1.2241855146681304


## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalized to your model to improve performance.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
tfidfVectorizer = TfidfVectorizer(stop_words=stop_words_lemma,
                                 tokenizer=lemmatizer)

In [15]:
normalized_model = Pipeline([
    ('extract', ColumnSelectTransformer(key)),
    ('tfidfVec', tfidfVectorizer),
    ()
    ('ridge', Ridge())
])

In [16]:
#normalized_model.fit(data, overall_array)

In [27]:
grader.score.nlp__normalized_model(bigrams_model.predict)

Your score:  0.9058483377389017


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [18]:
data_tiny = data[0:41]
overall_array_tiny = overall_array[0:41]

In [19]:
data_half = data[0:40001]
overall_array_half = overall_array[0:40001]
#Your score:  0.7388217686014964

In [20]:
data_fifteen = data[0:15001]
overall_array_fifteen = overall_array[0:15001]

In [84]:
tfidfVectorizer2 = TfidfVectorizer(stop_words=STOP_WORDS)

In [22]:
# from sklearn.model_selection import GridSearchCV

# paramgrid = {'alpha': [2,5,10,15,20],
#              'fit_intercept': [True, False],
#             }

# gs = GridSearchCV(Ridge(),
#                  param_grid=paramgrid,
#                  cv=3,
#                  n_jobs=2,
#                  verbose=1)

In [23]:
bigrams_model = Pipeline([
    ('extract', ColumnSelectTransformer(key)),
    ('tfidfVec', tfidfVectorizer2),
    ('ridge', Ridge(alpha=2))
])

In [24]:
bigrams_model.fit(data, overall_array)

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-pron-'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('extract', ColumnSelectTransformer(key='reviewText')),
                ('tfidfVec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={...
                                             'any', 'anyhow', 'anyone',
                                             'anything', 'anyway', 'anywhere',
                                             'around', ...},
                                 strip_accents=None, sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',


In [109]:
#bigrams_model.named_steps.gs.best_params_

In [26]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score:  0.8703913777152074


## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the review is negative given the word "terrible" appears in the text. Using these probabilities, we can derive a polarity score for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should
1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting and include stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [29]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

File ‘./data/amazon_one_and_five_star_reviews.json.gz’ already there; not retrieving.



To avoid memory issue, we can delete the older data.

In [32]:
del overall_array

In [33]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = np.asarray([dictionary.get("overall") for dictionary in data_polarity])

In [47]:
ratings.shape

(10000,)

In [110]:
from sklearn.naive_bayes import MultinomialNB

In [111]:
from sklearn.model_selection import GridSearchCV

paramgrid = {'alpha': range(1,11,1)}

gs = GridSearchCV(MultinomialNB(),
                 param_grid=paramgrid,
                 cv=3,
                 n_jobs=2,
                 verbose=1)

In [119]:
mnb_model = Pipeline([
    ('extract', ColumnSelectTransformer(key)),
    ('tfidfVec', tfidfVectorizer2),
    ('mnb', MultinomialNB())
])

In [120]:
mnb_model.fit(data_polarity, ratings)

Pipeline(memory=None,
         steps=[('extract', ColumnSelectTransformer(key='reviewText')),
                ('tfidfVec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={...
                                             'alone', 'along', 'already',
                                             'also', 'although', 'always', 'am',
                                             'among', 'amongst', 'amount', 'an',
                                             'and', 'another', 'any', 'anyhow',
                                     

In [121]:
#mnb_model.named_steps.gs.best_params_

In [131]:
import pandas as pd
df = pd.DataFrame(mnb_model.named_steps.mnb.feature_log_prob_).transpose()

In [134]:
df['polarity'] = df[1]/df[0]

In [136]:
df.head()

,0,1,polarity
0,-8.645958,-9.352072,1.081670
1,-9.636902,-9.302883,0.965340
2,-10.864393,-10.572737,0.973155
3,-10.864393,-10.574249,0.973294
4,-10.383420,-10.817837,1.041838


In [141]:
bottom_25 = df.sort_values(by='polarity')[0:25]  

In [142]:
bottom_25.head()

,0,1,polarity
11288,-9.388391,-7.041491,0.750021
16559,-9.028168,-6.849281,0.758657
3610,-10.112543,-7.782027,0.769542
10685,-7.155090,-5.616999,0.785035
8880,-8.796707,-6.939699,0.788897


In [143]:
top_25 = df.sort_values(by='polarity', ascending=False)[0:25]

In [144]:
top_25.head()

,0,1,polarity
24531,-6.956025,-9.923249,1.426569
19021,-6.866012,-9.739454,1.418502
18497,-7.682305,-10.817837,1.408150
19024,-7.616636,-10.415972,1.367529
12809,-7.209866,-9.736306,1.350414


In [145]:
index_list = []
for index, row in top_25.iterrows():
    index_list.append(index)

In [146]:
for index, row in bottom_25.iterrows():
    index_list.append(index)

In [147]:
index_list[0:5]

[24531, 19021, 18497, 19024, 12809]

In [148]:
list_of_feature_names = mnb_model.named_steps.tfidfVec.get_feature_names()

In [149]:
top_50 = []
for index in index_list:
    top_50.append(list_of_feature_names[index])

In [150]:
top_50

['waste',
 'return',
 'refund',
 'returning',
 'junk',
 'returned',
 'worst',
 'terrible',
 'useless',
 'garbage',
 'poor',
 'worse',
 'defective',
 'worthless',
 'support',
 'trash',
 'stopped',
 'sent',
 'beware',
 'horrible',
 'send',
 'tech',
 'dead',
 'awful',
 'unacceptable',
 'highly',
 'perfect',
 'beat',
 'great',
 'excellent',
 'lens',
 'amazing',
 'protects',
 'easy',
 'sturdy',
 'monopod',
 'macro',
 'pleased',
 'portrait',
 'love',
 'incredible',
 'lenses',
 'awesome',
 'charm',
 'price',
 'handy',
 'bokeh',
 'fits',
 'job',
 'photography']

In [151]:
grader.score.nlp__most_polar(top_50)

Your score:  0.9200000000000005


## Question 5: Topic modeling [optional]

Topic modeling is the analysis of determining the key topics or themes in a corpus. With respect to machine learning, topic modeling is an unsupervised technique. One way to uncover the main topics in a corpus is to use [non-negative matrix factorization](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html). For this question, use non-negative matrix factorization to determine the top ten words for the first twenty topics. You should submit your answer as a list of lists. What topics exist in the reviews?

In [ ]:
from sklearn.decomposition import NMF
 

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*